<a href="https://colab.research.google.com/github/salemsama/nn-zero-to-hero/blob/main/makemore1/makemore_part1_E02_splitdataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#E02: split up the dataset randomly into 80% train set, 10% dev set, 10% test set.
#Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?

# result:
# i see that all of them have diffrent loss
# i did it only on the bigram model becuase the trigram take a long time to train on my potato pc


import torch
import torch.nn.functional as F

In [ ]:
words = open("names.txt", 'r').read().splitlines()


chars = sorted(list(set(''.join(words))))       # all letters, set used first because it doesn't allow duplicate
stoi = {s:i+1 for i,s in enumerate(chars)}      # dict to map string to int,    start from 1, 0 reserved for next line
stoi['.'] = 0                                   # special letter to mark start and end of word
itos = {i:s for s,i in stoi.items()}            # dict to map int to string


In [ ]:
import random
random.shuffle(words)
splits = [round(len(words)*0.8),round(len(words)*0.9)]
train = words[:splits[0]]
dev   = words[splits[0]:splits[1]]
test  = words[splits[1]:]
print(f"train: {train}\ndev: {dev}\ntest: {test}")

train: ['sabrina', 'kayne', 'jermani', 'quincee', 'alianah', 'nicolai', 'baird', 'bethanny', 'arlyn', 'delailah', 'vanessa', 'yasna', 'wrenley', 'sarika', 'julyana', 'lanea', 'kiran', 'carpenter', 'ira', 'carmelita', 'ayira', 'kausar', 'jaylanni', 'lauralee', 'hian', 'jovee', 'lina', 'mekai', 'baker', 'jadore', 'hyrum', 'nyjah', 'anjay', 'seva', 'holten', 'omid', 'jayahna', 'arshaan', 'erika', 'ashtin', 'ward', 'yefim', 'sai', 'erielle', 'forever', 'dasani', 'warrick', 'reef', 'jasper', 'cheston', 'yanxi', 'mitt', 'taylore', 'herminio', 'aleksia', 'caoimhe', 'aizlynn', 'emilija', 'brier', 'mysha', 'kaisha', 'tamjid', 'joseangel', 'haelynn', 'makeena', 'allianna', 'nyara', 'zoi', 'liona', 'solene', 'nida', 'cale', 'milaysia', 'arlie', 'thai', 'coryion', 'amylee', 'hetvik', 'jeydon', 'zalayah', 'kimani', 'babygirl', 'lun', 'orin', 'maksen', 'drayven', 'yaritzy', 'theryn', 'nefertari', 'ryett', 'kimia', 'kiros', 'dwain', 'ahava', 'murray', 'georgie', 'nyx', 'avella', 'zlatan', 'hayder', '

In [ ]:
# training set
tx, ty = [], [] # lists for first letters and following letters
for word in train:
    chs = ['.'] + list(word) + ['.']    # word was string now list of char and special char before and after
    for ch1,ch2 in zip(chs, chs[1:]):   # zip make a tuple of every couple of chs
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]                 # store the number of the char

        tx.append(ix1)                  # here map tx ans ty
        ty.append(ix2)                  # tx is the input and ty is the correct output

tx = torch.tensor(tx)
ty = torch.tensor(ty)
tnum = tx.nelement()
tnum

182628

In [ ]:
# initialize the 'network'
gen = torch.Generator().manual_seed(2147483647)
W = torch.randn((27,27), generator=gen, requires_grad=True) # 27*27 network with grad

In [ ]:
def grad_desc(n):
    for k in range(n):
        # forward pass
            # we reprsent every letter as one hot (arrays of 0 except 1 place of the letter index)
        xenc = F.one_hot(tx, num_classes=27).float() # xs encoded as noe hots
        #one hot for a for example
        #[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]

        # the input * the weights
        logits = xenc @ W # matrix mulipication in pytorch, logits for log-counts
        # we mulitply xenc wich is n*27 @ 27*27 it out put n*27
        # thats why W is 27*27 in order for the mulipication to be correct
        # if it was 27*1 it would give us n*1 output

        # non linearity, this two lines called softmax
        counts = logits.exp() # fake counts, when exp all numbers become positive kinda like counts
        probs = counts / counts.sum(1, keepdim=True) # normalizing the probability so the sum equals 1
        # the division here is 27*27 / 27*1 so every row is divided by its sum

        # the loss function
        loss= probs[torch.arange(tnum), ty] # here we collect the probabilities of the correct output ys
        loss= -loss.log().mean()    # here we used the negative liklyhood loss function
        loss = loss + 0.01 * (W**2).mean() # regularazition, like adding 1 to counts to there is no 0 count
        print(loss.item())

        # backward pass
        W.grad = None   #zerograd, None is more efficient than zero
        loss.backward() # set the grad to how much it influnce the loss
        W.data += -50*W.grad # we use the minus becuase we want to lower the loss

In [ ]:
grad_desc(100)

2.484387159347534
2.4843392372131348
2.484304904937744
2.484276056289673
2.4842491149902344
2.4842236042022705
2.484199285507202
2.484175682067871
2.4841530323028564
2.484130859375
2.4841091632843018
2.484088182449341
2.48406720161438
2.484046459197998
2.4840269088745117
2.484006881713867
2.483987331390381
2.4839680194854736
2.4839489459991455
2.4839305877685547
2.4839119911193848
2.4838943481445312
2.4838759899139404
2.483858346939087
2.4838411808013916
2.483823776245117
2.483806610107422
2.4837899208068848
2.483773708343506
2.4837570190429688
2.483741044998169
2.483725070953369
2.4837090969085693
2.4836935997009277
2.4836783409118652
2.483663320541382
2.4836483001708984
2.483633518218994
2.483618974685669
2.483604669570923
2.4835898876190186
2.4835760593414307
2.4835619926452637
2.483548402786255
2.483534812927246
2.4835214614868164
2.4835078716278076
2.483494997024536
2.4834821224212646
2.483469009399414
2.4834561347961426
2.4834437370300293
2.483431339263916
2.4834189414978027
2.48

In [ ]:
# dev set
dx, dy = [], [] # lists for first letters and following letters
for word in dev:
    chs = ['.'] + list(word) + ['.']    # word was string now list of char and special char before and after
    for ch1,ch2 in zip(chs, chs[1:]):   # zip make a tuple of every couple of chs
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]                 # store the number of the char

        dx.append(ix1)                  # here map dx ans dy
        dy.append(ix2)                  # dx is the input and dy is the correct output

dx = torch.tensor(dx)
dy = torch.tensor(dy)
dnum = dx.nelement()

def dev_grad_desc(n):
    for k in range(n):
        # forward pass
            # we reprsent every letter as one hot (arrays of 0 except 1 place of the letter index)
        xenc = F.one_hot(dx, num_classes=27).float() # xs encoded as noe hots
        #one hot for a for example
        #[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]

        # the input * the weights
        logits = xenc @ W # matrix mulipication in pytorch, logits for log-counts
        # we mulitply xenc wich is n*27 @ 27*27 it out put n*27
        # thats why W is 27*27 in order for the mulipication to be correct
        # if it was 27*1 it would give us n*1 output

        # non linearity, this two lines called softmax
        counts = logits.exp() # fake counts, when exp all numbers become positive kinda like counts
        probs = counts / counts.sum(1, keepdim=True) # normalizing the probability so the sum equals 1
        # the division here is 27*27 / 27*1 so every row is divided by its sum

        # the loss function
        loss= probs[torch.arange(dnum), dy] # here we collect the probabilities of the correct output ys
        loss= -loss.log().mean()    # here we used the negative liklyhood loss function
        loss = loss + 0.0001 * (W**2).mean() # regularazition, like adding 1 to counts to there is no 0 count
        print(loss.item())

        # backward pass
        W.grad = None   #zerograd, None is more efficient than zero
        loss.backward() # set the grad to how much it influnce the loss
        W.data += -1*W.grad # we use the minus becuase we want to lower the loss

In [ ]:
dev_grad_desc(10)

2.461660623550415
2.461641550064087
2.4616219997406006
2.4616029262542725
2.4615838527679443
2.4615652561187744
2.4615466594696045
2.4615280628204346
2.4615094661712646
2.461491107940674


In [ ]:
# test set
sx, sy = [], [] # lists for first letters and following letters
for word in test:
    chs = ['.'] + list(word) + ['.']    # word was string now list of char and special char before and after
    for ch1,ch2 in zip(chs, chs[1:]):   # zip make a tuple of every couple of chs
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]                 # store the number of the char

        sx.append(ix1)                  # here map dx ans dy
        sy.append(ix2)                  # dx is the input and dy is the correct output

sx = torch.tensor(sx)
sy = torch.tensor(sy)
snum = sx.nelement()

def test_grad_desc(n):
    for k in range(n):
        # forward pass
            # we reprsent every letter as one hot (arrays of 0 except 1 place of the letter index)
        xenc = F.one_hot(sx, num_classes=27).float() # xs encoded as noe hots
        #one hot for a for example
        #[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]

        # the input * the weights
        logits = xenc @ W # matrix mulipication in pytorch, logits for log-counts
        # we mulitply xenc wich is n*27 @ 27*27 it out put n*27
        # thats why W is 27*27 in order for the mulipication to be correct
        # if it was 27*1 it would give us n*1 output

        # non linearity, this two lines called softmax
        counts = logits.exp() # fake counts, when exp all numbers become positive kinda like counts
        probs = counts / counts.sum(1, keepdim=True) # normalizing the probability so the sum equals 1
        # the division here is 27*27 / 27*1 so every row is divided by its sum

        # the loss function
        loss= probs[torch.arange(snum), sy] # here we collect the probabilities of the correct output ys
        loss= -loss.log().mean()    # here we used the negative liklyhood loss function
        loss = loss + 0.01 * (W**2).mean() # regularazition, like adding 1 to counts to there is no 0 count
        print(loss.item())

        # backward pass
        W.grad = None   #zerograd, None is more efficient than zero
        loss.backward() # set the grad to how much it influnce the loss
        W.data += -1*W.grad # we use the minus becuase we want to lower the loss

In [ ]:
test_grad_desc(10)

2.4702470302581787
2.4702036380767822
2.470161199569702
2.470118761062622
2.4700777530670166
2.470036745071411
2.469996452331543
2.469956398010254
2.469917058944702
2.4698781967163086


In [ ]:
for i in range(5):
            out = []
            ix = 0
            while True:
                xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
                logits = xenc @ W
                counts = logits.exp()
                p = counts / counts.sum(1, keepdim=True)

                ix = torch.multinomial(p,num_samples=1,replacement=True,generator=gen).item()
                out.append(itos[ix])
                if ix == 0:
                    break
            print(''.join(out))

morvann.
akela.
az.
arileri.
chaiadayra.
